📂 1. Import Required Libraries

In [ ]:
# 📌 Import necessary libraries
import cv2
import numpy as np
import tensorflow as tf
import collections
import os
from tensorflow.keras.models import load_model

🧠 2. Load the Road Rage Model

In [ ]:
# 📌 Load the pre-trained road rage detection model
model = load_model('road_rage_detection_model2.h5')
print("Pretrained Road Rage Model Loaded Successfully!")

🎥 3. Live Road Rage Detection with Frame Saving

In [ ]:
# 📌 Function for real-time road rage detection & saving only violence frames
def live_road_rage_detection(model, video_path, output_dir, buffer_size=30, smoothing_window_size=5, violence_threshold=0.6):
    """
    Perform real-time road rage detection on a video stream and save frames labeled as violence.
    
    Parameters:
    - model: Pretrained deep learning model
    - video_path: Path to the video file (or use 0 for webcam)
    - output_dir: Directory to save frames with detected violence
    - buffer_size: Number of frames to process in a sequence for prediction
    - violence_threshold: Threshold for classifying violence (default = 0.6)
    """
    
    # 📌 Extract the video name without the extension
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # 📌 Create a directory for storing detected frames
    video_output_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_output_dir, exist_ok=True)

    # 📌 Initialize video capture
    video_capture = cv2.VideoCapture(video_path)

    frame_buffer = []  # Stores frames for batch processing
    predictions_buffer = collections.deque(maxlen=smoothing_window_size)  # Stores recent predictions
    frame_count = 0  # Track frame numbers
    stored_frame_indices = set()  # Keep track of saved frames to avoid duplicates

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_count += 1

        # 📌 Resize the frame to match model input size (150x150)
        frame_resized = cv2.resize(frame, (150, 150))
        frame_buffer.append(frame_resized)

        # 📌 Perform prediction once we have enough frames
        if len(frame_buffer) == buffer_size:
            input_data = np.array(frame_buffer).reshape((1, buffer_size, 150, 150, 3)) / 255.0
            prediction = model.predict(input_data)[0][0]

            # 📌 Apply threshold to determine if road rage violence is present
            is_violence = 1 if prediction > violence_threshold else 0
            predictions_buffer.append(is_violence)

            # 📌 Apply temporal smoothing: Majority voting within the buffer window
            if sum(predictions_buffer) > len(predictions_buffer) / 2:
                label = "Violence"
                color = (0, 0, 255)  # Red for violence

                # 📌 Save the frame if it hasn't been saved before
                if frame_count not in stored_frame_indices:
                    stored_frame_indices.add(frame_count)

                    # 📌 Add text label before saving the image
                    frame_with_label = frame.copy()
                    cv2.putText(frame_with_label, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)

                    # 📌 Save frame
                    frame_filename = os.path.join(video_output_dir, f"frame_{frame_count}.jpg")
                    cv2.imwrite(frame_filename, frame_with_label)
                    
            else:
                label = "No Violence"
                color = (0, 255, 0)  # Green for no violence

            # 📌 Remove the oldest frame from buffer (FIFO mechanism)
            frame_buffer.pop(0)

        # 📌 Display the label on the frame
        cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)

        # 📌 Show the live video feed with the detection label
        cv2.imshow('Road Rage Detection', frame)

        # 📌 Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 📌 Release resources
    video_capture.release()
    cv2.destroyAllWindows()

🎯 4. Run Live Road Rage Detection on a Video

In [ ]:
# 📌 Paths and parameters
video_path = 'sample2.mp4'  # Replace with your video path
output_dir = 'Violent_Frames'  # Directory to store detected frames

# 📌 Start live road rage detection and save violence frames
live_road_rage_detection(model, video_path, output_dir)